# Colunas e Linhas no Apache Spark

O Apache Spark oferece recursos robustos para trabalhar com colunas e linhas de dados. As colunas são componentes essenciais para selecionar e transformar dados, enquanto as linhas representam entradas individuais em um DataFrame.

## Colunas

As colunas são objetos que representam uma única coluna em um DataFrame. Elas possuem vários métodos e funcionalidades para transformação e manipulação de dados.

| Recurso                                      | Descrição                                                                                          |
|---------------------------------------------|----------------------------------------------------------------------------------------------------|
| Seleção de Coluna                           | Você pode selecionar uma coluna usando notação de ponto, como `df.coluna`, ou `df["coluna"]`.       |
| Operações Matemáticas                       | As colunas suportam operações matemáticas, como soma, subtração, multiplicação e divisão.        |
| Funções de Agregação                        | Você pode aplicar funções de agregação, como `count`, `sum`, `avg`, `max` e `min`, a uma coluna. |
| Alias (Apelidos)                            | É possível renomear colunas usando o método `.alias("novo_nome")`.                                |
| Operações de String                         | Realize operações de string em colunas usando funções como `.substr`, `.lower`, `.replace`, etc. |

## Linhas

As linhas representam entradas individuais em um DataFrame. Elas contêm os valores para cada coluna em um determinado registro.

| Recurso                                      | Descrição                                                                                      |
|---------------------------------------------|------------------------------------------------------------------------------------------------|
| Acesso a Dados                              | Você pode acessar os dados de uma linha usando notação de ponto ou índices, como `linha.coluna` ou `linha[índice]`. |
| Conversão para Dicionário                   | Converta uma linha em um dicionário para fácil manipulação usando o método `.asDict()`.    |
| Comparação de Linhas                        | Compare linhas para verificar igualdade ou ordem usando operadores de comparação.         |
| Construção de Linhas                        | Crie linhas usando o construtor `Row`. Isso é útil para criar novas linhas em operações de transformação de dados. |

## Exemplo de Uso

Aqui está um exemplo que demonstra a seleção de colunas e o acesso a dados de linhas em um DataFrame:

```python
# Importe os módulos necessários
from pyspark.sql import SparkSession, Row

# Crie um DataFrame de exemplo
data = [Row(nome="Alice", idade=30), Row(nome="Bob", idade=35)]
df = spark.createDataFrame(data)

# Selecione uma coluna
nomes = df.select("nome")

# Acesse dados da linha
primeira_linha = df.first()
nome_da_primeira_linha = primeira_linha.nome
```

In [4]:
from pyspark.sql import SparkSession, Row

spark = SparkSession.builder.appName("Exemplo").getOrCreate()

In [5]:
df = (
    spark
    .read
    .format("csv")
    .option("inferSchema","true")
    .option("encoding","utf-8")
    .option("delimiter",";") #ou sep
    .option("header", "true")
    .load("data/landing/VRA/")
)
df.printSchema()

root
 |-- ICAO Empresa Aerea: string (nullable = true)
 |-- Numero Voo: string (nullable = true)
 |-- Codigo DI: string (nullable = true)
 |-- Codigo Tipo Linha: string (nullable = true)
 |-- ICAO Aerodromo Origem: string (nullable = true)
 |-- ICAO Aerodromo Destino: string (nullable = true)
 |-- Partida Prevista: string (nullable = true)
 |-- Partida Real: string (nullable = true)
 |-- Chegada Prevista: string (nullable = true)
 |-- Chegada Real: string (nullable = true)
 |-- Situacao Voo: string (nullable = true)
 |-- Codigo Justificativa: string (nullable = true)



In [6]:
df[0]

Column<'ICAO Empresa Aerea'>

In [52]:
df.collect()

KeyboardInterrupt: 

In [7]:
df["Codigo Tipo Linha"]

Column<'Codigo Tipo Linha'>

In [8]:
df.Chegada Real

SyntaxError: invalid syntax (699309396.py, line 1)

In [9]:
df = df.withColumnRenamed('Chegada Real','Chegada_Real')
df.Chegada_Real

Column<'Chegada_Real'>

In [10]:
df = df.withColumn('Situacao_Voo',df['Situacao Voo'])

In [11]:
df.printSchema()

root
 |-- ICAO Empresa Aerea: string (nullable = true)
 |-- Numero Voo: string (nullable = true)
 |-- Codigo DI: string (nullable = true)
 |-- Codigo Tipo Linha: string (nullable = true)
 |-- ICAO Aerodromo Origem: string (nullable = true)
 |-- ICAO Aerodromo Destino: string (nullable = true)
 |-- Partida Prevista: string (nullable = true)
 |-- Partida Real: string (nullable = true)
 |-- Chegada Prevista: string (nullable = true)
 |-- Chegada_Real: string (nullable = true)
 |-- Situacao Voo: string (nullable = true)
 |-- Codigo Justificativa: string (nullable = true)
 |-- Situacao_Voo: string (nullable = true)



In [12]:
df_test = df.select(df.Chegada_Real.alias("dataway"))
df_test.printSchema()

root
 |-- dataway: string (nullable = true)



In [13]:
df.columns

['ICAO Empresa Aerea',
 'Numero Voo',
 'Codigo DI',
 'Codigo Tipo Linha',
 'ICAO Aerodromo Origem',
 'ICAO Aerodromo Destino',
 'Partida Prevista',
 'Partida Real',
 'Chegada Prevista',
 'Chegada_Real',
 'Situacao Voo',
 'Codigo Justificativa',
 'Situacao_Voo']

In [14]:
print(*[col.lower().replace(" ","_") for col in df.columns])

icao_empresa_aerea numero_voo codigo_di codigo_tipo_linha icao_aerodromo_origem icao_aerodromo_destino partida_prevista partida_real chegada_prevista chegada_real situacao_voo codigo_justificativa situacao_voo


In [14]:
df = df.toDF(*[col.lower().replace(" ","_") for col in df.columns])
df.printSchema()
df.show()

root
 |-- icao_empresa_aerea: string (nullable = true)
 |-- numero_voo: string (nullable = true)
 |-- codigo_di: string (nullable = true)
 |-- codigo_tipo_linha: string (nullable = true)
 |-- icao_aerodromo_origem: string (nullable = true)
 |-- icao_aerodromo_destino: string (nullable = true)
 |-- partida_prevista: string (nullable = true)
 |-- partida_real: string (nullable = true)
 |-- chegada_prevista: string (nullable = true)
 |-- chegada_real: string (nullable = true)
 |-- situacao_voo: string (nullable = true)
 |-- codigo_justificativa: string (nullable = true)
 |-- situacao_voo: string (nullable = true)

+------------------+----------+---------+-----------------+---------------------+----------------------+----------------+----------------+----------------+----------------+------------+--------------------+------------+
|icao_empresa_aerea|numero_voo|codigo_di|codigo_tipo_linha|icao_aerodromo_origem|icao_aerodromo_destino|partida_prevista|    partida_real|chegada_prevista|    ch

# Tipos de Dados no Apache Spark

O Apache Spark oferece uma variedade de tipos de dados que podem ser usados para representar informações em DataFrames. Cada tipo de dado tem características específicas que o tornam adequado para diferentes cenários.

## Tipos de Dados Primitivos

Os tipos de dados primitivos são os blocos de construção fundamentais para representar informações em Spark:

| Tipo de Dado   | Descrição                                      | Exemplo de Caso de Uso                     |
|-----------------|------------------------------------------------|------------------------------------------|
| IntegerType     | Números inteiros de 32 bits                     | Idades, quantidades, índices.             |
| LongType        | Números inteiros de 64 bits                     | IDs exclusivos, contagens extensas.      |
| FloatType       | Números de ponto flutuante de 32 bits           | Medidas com decimais, preços.             |
| DoubleType      | Números de ponto flutuante de 64 bits           | Valores de alta precisão, coordenadas.    |
| StringType      | Sequências de caracteres                        | Nomes, descrições, textos.                |
| BooleanType     | Valores booleanos (True/False)                 | Marcadores de verdadeiro ou falso.        |
| DateType        | Datas                                        | Datas de eventos, vencimentos.            |
| TimestampType   | Data e hora                                 | Registro de eventos com horários.        |
| BinaryType      | Dados binários                                | Imagens, arquivos binários.               |
| NullType        | Valor nulo (ausência de valor)                | Registros incompletos ou ausentes.        |

## Tipos de Dados Complexos

Os tipos de dados complexos permitem representar informações mais estruturadas:

| Tipo de Dado        | Descrição                                    | Exemplo de Caso de Uso                  |
|---------------------|----------------------------------------------|---------------------------------------|
| ArrayType           | Lista ordenada de elementos                  | Lista de produtos em um pedido.      |
| MapType             | Coleção de pares chave-valor                  | Dicionário de configurações.          |
| StructType          | Estrutura de dados com campos nomeados       | Registro de funcionário com nome e idade. |
| StructField         | Campo dentro de um StructType                | Colunas em um DataFrame.              |


In [ ]:
schema_json = df.schema.json()
ddl = spark.sparkContext._jvm.org.apache.spark.sql.types.DataType.fromJson(schema_json).toDDL()

ddl

In [15]:
from pyspark.sql.types import *

schema_method = StructType(
    [
        StructField('icao_empresa_aerea', StringType()),
        StructField('numero_voo', StringType()),
        StructField('codigo_di', StringType()),
        StructField('codigo_tipo_linha', StringType()),
        StructField('icao_aerodromo_origem', StringType()),
        StructField('icao_aerodromo_destino', StringType()),
        StructField('partida_prevista', StringType()),
        StructField('partida_real', StringType()),
        StructField('chegada_prevista', StringType()),
        StructField('chegada_real', StringType()),
        StructField('situacao_voo', StringType()),
        StructField('codigo_justificativa', StringType()),
    ]
)

In [16]:
df_final = (
    spark
    .read
    .format("csv")
    .option("inferSchema","true")
    .option("encoding","utf-8")
    .option("delimiter",";") #ou sep
    .option("header", "true")
    .schema(schema_method)
    .load("data/landing/VRA/")
)

df_final.printSchema()

root
 |-- icao_empresa_aerea: string (nullable = true)
 |-- numero_voo: string (nullable = true)
 |-- codigo_di: string (nullable = true)
 |-- codigo_tipo_linha: string (nullable = true)
 |-- icao_aerodromo_origem: string (nullable = true)
 |-- icao_aerodromo_destino: string (nullable = true)
 |-- partida_prevista: string (nullable = true)
 |-- partida_real: string (nullable = true)
 |-- chegada_prevista: string (nullable = true)
 |-- chegada_real: string (nullable = true)
 |-- situacao_voo: string (nullable = true)
 |-- codigo_justificativa: string (nullable = true)



In [ ]:
df_final.show()

# Vra Bronze save

In [17]:
(
    df_final
    .write
    .format("parquet")
    .mode("append")
    .save("/home/app/data/1.bronze/vra/")
)

# Aerodromos Bronze save

In [ ]:
(
    df_distincts
    .write
    .format("parquet")
    .mode("append")
    .save("/home/app/data/1.bronze/aerodromos/")
)